# अध्याय ७: च्याट अनुप्रयोगहरू निर्माण गर्दै
## Azure OpenAI API छिटो सुरु गर्ने तरीका


## अवलोकन
यो नोटबुक [Azure OpenAI नमूना रिपोजिटरी](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) बाट अनुकूलित गरिएको हो जसमा नोटबुकहरू समावेश छन् जसले [OpenAI](notebook-openai.ipynb) सेवा पनि पहुँच गर्छन्।

Python OpenAI API Azure OpenAI सँग पनि काम गर्छ, केही परिमार्जनहरूसहित। यहाँ थप जान्नुहोस्: [Python सँग OpenAI र Azure OpenAI अन्तबिन्दुहरू बीच कसरी स्विच गर्ने](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

थप छिटो सुरुका उदाहरणहरूको लागि कृपया आधिकारिक [Azure OpenAI छिटो सुरु दस्तावेज](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।


## Table of Contents  

[Overview](../../../../07-building-chat-applications/python)  
[Getting started with Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[Build your first prompt](../../../../07-building-chat-applications/python)  

[Use Cases](../../../../07-building-chat-applications/python)    
[1. Summarize Text](../../../../07-building-chat-applications/python)  
[2. Classify Text](../../../../07-building-chat-applications/python)  
[3. Generate New Product Names](../../../../07-building-chat-applications/python)  
[4. Fine Tune a Classifier](../../../../07-building-chat-applications/python)  
[5. Embeddings](../../../../07-building-chat-applications/python)

[References](../../../../07-building-chat-applications/python)


### Azure OpenAI सेवा सुरु गर्ने तरिका

नयाँ ग्राहकहरूले Azure OpenAI सेवामा पहुँचको लागि [आवेदन गर्नुपर्नेछ](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)।  
स्वीकृति पूरा भएपछि, ग्राहकहरूले Azure पोर्टलमा लगइन गरी Azure OpenAI सेवा स्रोत सिर्जना गर्न सक्छन् र स्टुडियो मार्फत मोडेलहरूसँग प्रयोग गर्न सुरु गर्न सक्छन्।  

[छिटो सुरु गर्नको लागि उत्कृष्ट स्रोत](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### आफ्नो पहिलो प्रॉम्प्ट बनाउनुहोस्  
यो छोटो अभ्यासले "सारांश" नामक सरल कार्यका लागि OpenAI मोडेलमा प्रॉम्प्टहरू पेश गर्ने आधारभूत परिचय प्रदान गर्नेछ।


**चरणहरू**:  
1. आफ्नो पायथन वातावरणमा OpenAI लाइब्रेरी स्थापना गर्नुहोस्  
2. मानक सहायक लाइब्रेरीहरू लोड गर्नुहोस् र तपाईंले सिर्जना गरेको OpenAI सेवा लागि तपाईंको सामान्य OpenAI सुरक्षा प्रमाणपत्रहरू सेट गर्नुहोस्  
3. तपाईंको कार्यका लागि मोडेल चयन गर्नुहोस्  
4. मोडेलका लागि सरल प्रॉम्प्ट सिर्जना गर्नुहोस्  
5. मोडेल API मा तपाईंको अनुरोध पेश गर्नुहोस्!


### १. OpenAI स्थापना गर्नुहोस्


  > [!NOTE] यदि यो नोटबुक Codespaces मा वा Devcontainer भित्र चलाइन्छ भने यो चरण आवश्यक छैन।


In [ ]:
%pip install openai python-dotenv

### २. सहायक पुस्तकालयहरू आयात गर्नुहोस् र प्रमाणपत्रहरू स्थापना गर्नुहोस्


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. सही मोडेल फेला पार्ने  
GPT-3.5-turbo वा GPT-4 मोडेलहरूले प्राकृतिक भाषा बुझ्न र उत्पन्न गर्न सक्छन्। सेवा चार मोडेल क्षमताहरू प्रदान गर्दछ, प्रत्येकमा विभिन्न स्तरको शक्ति र गति हुन्छ जुन विभिन्न कार्यहरूका लागि उपयुक्त हुन्छ। 

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. प्रॉम्प्ट डिजाइन  

"ठूला भाषा मोडेलहरूको जादू यो हो कि विशाल मात्रामा पाठमा यो पूर्वानुमान त्रुटि कम गर्न तालिम दिइएपछि, मोडेलहरूले यी पूर्वानुमानहरूका लागि उपयोगी अवधारणाहरू सिक्छन्। उदाहरणका लागि, तिनीहरूले यस्तो अवधारणाहरू सिक्छन्"(1):

* कसरी वर्तनी गर्ने
* कसरी व्याकरण काम गर्छ
* कसरी पुनःव्याख्या गर्ने
* कसरी प्रश्नहरूको उत्तर दिने
* कसरी संवाद राख्ने
* कसरी धेरै भाषाहरूमा लेख्ने
* कसरी कोड गर्ने
* आदि।

#### ठूला भाषा मोडेललाई कसरी नियन्त्रण गर्ने  
"ठूला भाषा मोडेलमा सबै इनपुटहरू मध्ये सबैभन्दा प्रभावकारी भनेको पाठ प्रॉम्प्ट हो(1)।

ठूला भाषा मोडेलहरूलाई केही तरिकाले आउटपुट उत्पादन गर्न प्रॉम्प्ट गर्न सकिन्छ:

- निर्देशन: मोडेललाई तपाईं के चाहनुहुन्छ भन्नुहोस्
- पूर्ति: मोडेललाई तपाईंले चाहेको कुरा सुरुबाट पूरा गर्न प्रेरित गर्नुहोस्
- प्रदर्शन: मोडेललाई तपाईं के चाहनुहुन्छ देखाउनुहोस्, जसमा:
- प्रॉम्प्टमा केही उदाहरणहरू
- फाइन-ट्युनिङ तालिम डेटासेटमा सयौं वा हजारौं उदाहरणहरू



#### प्रॉम्प्टहरू सिर्जना गर्न तीन आधारभूत दिशानिर्देशहरू छन्:

**देखाउनुहोस् र भन्नुहोस्**। तपाईं के चाहनुहुन्छ स्पष्ट पार्नुहोस्, निर्देशनहरू, उदाहरणहरू, वा दुवैको संयोजनमार्फत। यदि तपाईं मोडेललाई वस्तुहरूको सूचीलाई वर्णानुक्रममा क्रमबद्ध गर्न वा अनुच्छेदलाई भावना अनुसार वर्गीकरण गर्न चाहनुहुन्छ भने, यसो गर्न चाहनु भएको कुरा देखाउनुहोस्।

**गुणस्तरीय डेटा प्रदान गर्नुहोस्**। यदि तपाईं वर्गीकरणकर्ता बनाउन खोज्दै हुनुहुन्छ वा मोडेललाई कुनै ढाँचालाई पालना गर्न चाहनुहुन्छ भने, पर्याप्त उदाहरणहरू सुनिश्चित गर्नुहोस्। तपाईंका उदाहरणहरूलाई प्रमाणित गर्न निश्चित हुनुहोस् — मोडेल सामान्यतया आधारभूत वर्तनी त्रुटिहरू देख्न पर्याप्त बुद्धिमान हुन्छ र तपाईंलाई प्रतिक्रिया दिन्छ, तर यसले यो जानाजानी भएको ठान्न सक्छ र यसले प्रतिक्रियामा प्रभाव पार्न सक्छ।

**तपाईंका सेटिङहरू जाँच गर्नुहोस्।** तापक्रम र top_p सेटिङहरूले मोडेलले प्रतिक्रिया उत्पादन गर्दा कति निर्धारक हुन्छ भन्ने नियन्त्रण गर्छ। यदि तपाईंले यस्तो प्रतिक्रिया माग्दै हुनुहुन्छ जहाँ केवल एक सही उत्तर छ भने, तपाईंले यी सेटिङहरू कम राख्न चाहनुहुन्छ। यदि तपाईं विविध प्रतिक्रियाहरू खोज्दै हुनुहुन्छ भने, तपाईंले यी उच्च राख्न चाहनुहुन्छ। यी सेटिङहरूसँग मानिसहरूले गर्ने सबैभन्दा ठूलो गल्ती भनेको यीलाई "चतुराई" वा "रचनात्मकता" नियन्त्रणहरू ठान्नु हो।


स्रोत: https://learn.microsoft.com/azure/ai-services/openai/overview


तस्वीर तपाईंको पहिलो पाठ प्रॉम्प्ट सिर्जना गर्दैछ!


### ५. पेश गर्नुहोस्!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### उही कल दोहोर्याउनुहोस्, परिणामहरू कसरी तुलना हुन्छन्?


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Summarize Text  
#### Challenge  
पाठलाई 'tl;dr:' थपेर सारांश बनाउनुहोस्। कसरी मोडेलले कुनै अतिरिक्त निर्देशन बिना धेरै कार्यहरू कसरी गर्ने बुझ्छ भन्ने कुरामा ध्यान दिनुहोस्। तपाईं tl;dr भन्दा बढी वर्णनात्मक प्रॉम्प्टहरूसँग मोडेलको व्यवहार परिवर्तन गर्न र प्राप्त सारांशलाई अनुकूलित गर्न प्रयोग गर्न सक्नुहुन्छ(3)।  

भर्खरको कामले ठूलो पाठको संग्रहमा पूर्व-प्रशिक्षण गरेर र त्यसपछि विशिष्ट कार्यमा फाइन-ट्यून गरेर धेरै NLP कार्यहरू र मापनहरूमा महत्वपूर्ण प्रगति देखाएको छ। सामान्यतया कार्य-निरपेक्ष संरचनामा भए तापनि, यस विधिले अझै हजारौं वा दशौं हजार उदाहरणहरूको कार्य-विशिष्ट फाइन-ट्यूनिङ डेटासेटहरू आवश्यक पर्छ। यसको विपरीत, मानिसहरू सामान्यतया केही उदाहरणहरू वा सरल निर्देशनहरूबाट नयाँ भाषा कार्य गर्न सक्छन् - जुन वर्तमान NLP प्रणालीहरूले अझै ठूलो मात्रामा गर्न संघर्ष गरिरहेका छन्। यहाँ हामी देखाउँछौं कि भाषा मोडेलहरूलाई ठूलो बनाउँदा कार्य-निरपेक्ष, थोरै-शट प्रदर्शनमा ठूलो सुधार हुन्छ, कहिलेकाहीं पहिलेको अत्याधुनिक फाइन-ट्यूनिङ विधिहरूसँग प्रतिस्पर्धा गर्न सक्ने स्तरसम्म पुग्छ।  

Tl;dr


# विभिन्न प्रयोगका लागि अभ्यासहरू  
1. पाठ सारांश गर्नुहोस्  
2. पाठ वर्गीकरण गर्नुहोस्  
3. नयाँ उत्पादन नामहरू सिर्जना गर्नुहोस्  
4. एम्बेडिङहरू  
5. वर्गीकरणकर्ता फाइन ट्युन गर्नुहोस्


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## पाठ वर्गीकरण  
#### चुनौती  
आइटमहरूलाई अनुमान लगाउने समयमा प्रदान गरिएका वर्गहरूमा वर्गीकृत गर्नुहोस्। तलको उदाहरणमा, हामीले वर्गहरू र वर्गीकरण गर्नुपर्ने पाठ दुवैलाई प्रॉम्प्ट(*playground_reference) मा प्रदान गरेका छौं। 

ग्राहक सोधपुछ: नमस्ते, मेरो ल्यापटपको किबोर्डको एउटा कुञ्जी भर्खरै फुट्यो र मलाई प्रतिस्थापन चाहिन्छ:

वर्गीकृत वर्ग:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## नयाँ उत्पादन नामहरू सिर्जना गर्नुहोस्
#### चुनौती
उत्पादन नामहरू उदाहरण शब्दहरूबाट सिर्जना गर्नुहोस्। यहाँ हामीले नामहरू सिर्जना गर्न लागेका उत्पादनको बारेमा जानकारी प्रॉम्प्टमा समावेश गरेका छौं। हामीले प्राप्त गर्न चाहेको ढाँचालाई देखाउन समान उदाहरण पनि प्रदान गरेका छौं। हामीले उच्च तापमान मान सेट गरेका छौं जसले अनियमितता र थप नवप्रवर्तनात्मक प्रतिक्रियाहरू बढाउँछ।

उत्पादन विवरण: घरमा प्रयोग हुने मिल्कशेक मेकर  
बीउ शब्दहरू: छिटो, स्वस्थ, सानो।  
उत्पादन नामहरू: HomeShaker, Fit Shaker, QuickShake, Shake Maker

उत्पादन विवरण: कुनै पनि खुट्टाको आकारमा फिट हुने जुत्ताको जोडी।  
बीउ शब्दहरू: अनुकूलनीय, फिट, ओम्नी-फिट।


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Embeddings
यस खण्डले कसरी embeddings प्राप्त गर्ने र शब्दहरू, वाक्यहरू, र कागजातहरू बीच समानताहरू पत्ता लगाउने देखाउनेछ। तलका नोटबुकहरू चलाउनको लागि तपाईँले `text-embedding-ada-002` लाई आधार मोडेलको रूपमा प्रयोग गर्ने मोडेल तैनाथ गर्नुपर्नेछ र उसको तैनाथ नाम .env फाइल भित्र `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT` भेरिएबल प्रयोग गरी सेट गर्नुपर्नेछ।


### मोडेल वर्गीकरण - एक एम्बेडिङ मोडेल छनौट गर्दै

**मोडेल वर्गीकरण**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (एम्बेडिङ परिवार)  
{capability} --> ada             (सबै अन्य एम्बेडिङ मोडेलहरू २०२४ मा अवसान हुनेछन्)  
{input-type} --> n/a             (खोज मोडेलहरूका लागि मात्र निर्दिष्ट)  
{identifier} --> 002             (संस्करण ००२)  

model = 'text-embedding-ada-002'


  > [!NOTE] यदि यो नोटबुक Codespaces मा वा Devcontainer भित्र चलाइन्छ भने तलको चरण आवश्यक छैन।


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## cnn दैनिक समाचार डेटासेटबाट लेख तुलना गर्दै
स्रोत: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Azure Documentation - Azure OpenAI Models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# थप सहयोगको लागि  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# योगदानकर्ताहरू
* Louis Li  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज यसको मूल भाषामा आधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
